## The structure of languages

We will see in detail today the code for a lispy calculator we'll call `stupidlang`, focusing on environment frames as opposed to parsing. The parsing code is provided here, we shall talk about it later.

This interpreter essentionally reproduces a lisp interpreter from Peter Norvig, but stripped down, and simpler.  Our reasoning for doing this is to expose you to some features of python, the use of closures to develop objects with state, and to see some basics of how a language works.

Here is a program in this language:


```
(def rad 5)
(def radiusfunc (func (radius) (* pi (* radius radius))))
(radiusfunc rad)
(def myvar 0)
(if (== myvar 1) (store rad 6) (store rad 7))
(radiusfunc rad)
(== 1 1)
```


### Environment Implementation

Our initial implementation of this language will be rough and tumble, but we'll separate it into different files, separate interface from implementation, and package it up nicely as the semester goes...

Indeed we will replace some of our closure-based implementations here by classes.

#### Nested Environment Frames

We implement an environment as frames nested, with the outer(upper) environment captured as a closure.


In [ ]:
def frame(outerenv = None):
    bindings={}
    def lookup(variable):
        try:
            found = bindings[variable]
            env = dispatch
        except KeyError:#not found inside, so go to the outer
            if outerenv is not None:
                found, env = outerenv('lookup', variable)
            else:
                raise NameError("{} <<>> not found in Environment".format(variable))
        return found, env
    def extend(variable, value):
        bindings[variable] = value
    def extend_many(envtuples): # update can use a list of k,v tuples
        bindings.update(envtuples)
    def printit(): # for debugging
        return bindings
    #The dispatch function is what is returned
    def dispatch(message, variable=None, value=None):
        if message == 'lookup':
            return lookup(variable)
        elif message == 'extend':
            return extend(variable, value)
        elif message == 'extend_many':
            return extend_many(value)
        elif message == 'printit':
            return printit()
    return dispatch


Create a simple frame to test:

In [ ]:
tryenv=frame()
tryenv("extend", 'a', 5)
tups=[('b', 1), ('c', 2)]
tryenv("extend_many", None, tups)
tryenv('printit')

{'a': 5, 'b': 1, 'c': 2}

extend that frame with a child...

In [ ]:
try2env=frame(tryenv)
tryenv("extend", 'd', 55)
try2env("lookup",'d')[0], try2env("lookup",'a')[0]

(55, 5)

### Parser

We wont say much about the parser here except to remark that `lex` splits the program code into tokens, and converts to appropriate types using `typer`, and then `syn` converts these tokens into a nested list structure which reflects the structure of our language:

```python
program = """
(def rad 5)
rad
(def radiusfunc (func (radius) (* pi (* radius radius))))
(radiusfunc rad)
(def myvar 0)
(if (== myvar 1) (store rad 6) (store rad 7))
(radiusfunc rad)
(== 1 1)
"""
```

Line by line parse output:

```
[]
['def', 'rad', 5]
rad
['def', 'radiusfunc', ['func', ['radius'], ['*', 'pi', ['*', 'radius', 'radius']]]]
['radiusfunc', 'rad']
['def', 'myvar', 0]
['if', ['==', 'myvar', 1], ['store', 'rad', 6], ['store', 'rad', 7]]
['radiusfunc', 'rad']
['==', 1, 1]
[]
```

Nore that the parsing process and the code there has nothing to do with the execution environment. We'll see this later with how python is processed as well.

In [ ]:
Symbol = str

def typer(token):
    if token == 'true':
        return True
    elif token == 'false':
        return False
    try:
        t = int(token)
        return t
    except ValueError:
        try:
            t = float(token)
            return t
        except ValueError:
            return Symbol(token)

def lex(loc):
    tokenlist =  loc.replace('(', ' ( ').replace(')', ' ) ').split()
    return [typer(t) for t in tokenlist]

def syn(tokens):
    if len(tokens) == 0:
        return []
    token = tokens.pop(0)
    if token == '(':
        L = []
        while tokens[0] != ')':
            L.append(syn(tokens))
        tokens.pop(0) # pop off ')'
        return L
    else:
        if token==')':
            assert 1, "should not have got here"
        return token

def parse(loc):
    return syn(lex(loc))

### Evaluator

Now lets talk about the program evaluation. Our evaluator uses the python environment. While it does not define a python frame-stack, it uses recursion via nested environment functions in python. All functionality like ops and built-in funcs are outsourced to python.

This makes our language a **DSL**, or **Domain Specific Language**. Writing a "self-hosting" language is beyond the parameters of this course, but little DSL's using this kind of hosted structure, or even simpler, directly using syntax in the host language are very common. Examples are jquery in javascript, rails in ruby, etc...

We first define the top level:

#### Global Environment/Frame

In this environment we put in everything in the math library, and define ops in terms of the built-in python ops and functions, so that we have a reasonably functioning calculator...

We'll be importing everything from python's math module to use here...

In [ ]:
import math
vars(math)

{'__name__': 'math',
 '__doc__': 'This module provides access to the mathematical functions\ndefined by the C standard.',
 '__package__': '',
 '__loader__': _frozen_importlib.BuiltinImporter,
 '__spec__': ModuleSpec(name='math', loader=<class '_frozen_importlib.BuiltinImporter'>, origin='built-in'),
 'acos': <function math.acos(x, /)>,
 'acosh': <function math.acosh(x, /)>,
 'asin': <function math.asin(x, /)>,
 'asinh': <function math.asinh(x, /)>,
 'atan': <function math.atan(x, /)>,
 'atan2': <function math.atan2(y, x, /)>,
 'atanh': <function math.atanh(x, /)>,
 'ceil': <function math.ceil(x, /)>,
 'copysign': <function math.copysign(x, y, /)>,
 'cos': <function math.cos(x, /)>,
 'cosh': <function math.cosh(x, /)>,
 'degrees': <function math.degrees(x, /)>,
 'dist': <function math.dist(p, q, /)>,
 'erf': <function math.erf(x, /)>,
 'erfc': <function math.erfc(x, /)>,
 'exp': <function math.exp(x, /)>,
 'expm1': <function math.expm1(x, /)>,
 'fabs': <function math.fabs(x, /)>,
 'fact

In [ ]:
import math, operator as op
def global_frame():
    "An environment with some Scheme standard procedures."
    env = frame()
    env("extend_many", None, vars(math))
    env("extend_many", None, {
        '+':op.add, '-':op.sub, '*':op.mul, '/':op.truediv,
        'abs':     abs,
        'max':     max,
        'min':     min,
        'round':   round,
        '>':op.gt, '<':op.lt, '>=':op.ge, '<=':op.le, '==':op.eq,
        'not':     op.not_
    })
    return env

In [ ]:
globenv = global_frame()

#### The workhorse

The workhorse of our language is the `eval_ptree` below. What it does is to go one-by-one through the various types of symbols that arise, and evaluate them in the context of our environment and enclosing environment. It needs to work recursively, evalusting subexpressions in stuff like `if` statements and stupidlang function bodies.

In [ ]:
def eval_ptree(x, env):
    truthy_map={'#t':True, '#f':False, 'nil':None}
    if x in ('#t', '#f', 'nil'):#handle boolean tokens first
        return truthy_map[x]
    elif isinstance(x, Symbol): #else do a lookup
        # variable, op lookup
        return env("lookup", x)[0]
    elif not isinstance(x, list):  # if still not a list, we are a constant
        return x
    elif len(x)==0: #noop for an empty list
        return None
    elif x[0]=='if':#an if statement
        (_, predicate, truexpr, falseexpr) = x
        if eval_ptree(predicate, env):
            expression = truexpr
        else:
            expression = falseexpr
        return eval_ptree(expression, env)
    elif x[0] == 'def':         # variable or function definition, local
        #print('in def x is',x)
        (_, var, expression) = x
        #postorder traversal by nested eval is needed below
        env('extend', var, eval_ptree(expression, env))
    elif x[0] == 'store':           # (store var exp), like set!
        (_, var, exp) = x
        env_found_in = env("lookup", var)[1]#can be found in an outer env
        env_found_in("extend", var, eval_ptree(exp, env))
    elif x[0] == 'func': #this is the function definition, not the execution
        #print("in func x is",x)
        (_, parameters, parsedbody) = x
        return func(parameters, parsedbody, env)
    else:                          # operators, funcs calling
        #print("x", x, "env", env("printit"))
        op = eval_ptree(x[0], env)
        #postorder traversal to get subexpressione before running the op
        args = tuple([eval_ptree(arg, env) for arg in x[1:]])
        #print('argies', args, op.__name__)
        #Function execution
        if op.__name__=='dispatch':#need to handle our defined funcs
            return op('call', *args)
        else:#regular ops and funcs we added to the environment
            return op(*args)

#### Defining a function in our language

Here is where we define a function. The function object represented by the returned `dispatch` function uses a closure to hold in the params, env in which function was defined, and code body.

The actual execution happens when `call` is called, via a function call `eval_ptree` defined above.

In [ ]:
def func(params, parsedbody, env):
    def call(argstuple):
        print("in call", params, parsedbody, argstuple)
        # here is where we create a frame to run the function
        funcenv = frame(outerenv=env)
        funcenv('extend_many', None, zip(params, argstuple))
        #print(funcenv('printit'))
        return eval_ptree(parsedbody, funcenv)
    def dispatch(message, *args):
        print("in dispatch args are", message, args)
        if message=='call':
            return call(args)
    #print("at define time", params, parsedbody, env('printit'))
    return dispatch


In [ ]:
dafunc=func(['radius', 'area'],[], globenv)
dafunc('call', 5, 7)

in dispatch args are call (5, 7)
in call ['radius', 'area'] [] (5, 7)


### Driver to run the code

We provide some driver functions:

In [ ]:
def parse_program(program):
    "parse program line by line"
    output=[]
    program = [e.strip() for e in program.split('\n')]
    for l in program:
        output.append(parse(l))
    return output

In [ ]:
def run_program(program, env):
    """
    run program line by line, accumulating python
    output in an array
    """
    output=[]
    program = [e.strip() for e in program.split('\n')]
    # your code here
    for l in program:
        parsed = parse(l)
        print(">", parsed)
        output.append(eval_ptree(parsed, env))
    return output

The `backtolang` function below converts python output from stupidlang code snippets back into stupidlang, so appropriate outputs can be printed. The game is simply to stringify numbers, and convert bools and lists appropriately.

We also provide a `repl`, which allows us to run code line by line.

In [ ]:
def backtolang(exp):
    boolmap={True:'#t', False:'#f'}
    if  isinstance(exp, list):
        return '(' + ' '.join(map(backtolang, exp)) + ')'
    elif isinstance(exp, bool):
        return boolmap[exp]
    elif exp is None:
        return 'nil'
    else:
        return str(exp)

def repl(env, prompt='calc> '):
    while True:
        try:
            val = eval_ptree(parse(input(prompt)), env)
        except (KeyboardInterrupt, EOFError):
            break
        if val is not None:
            print(backtolang(val))



### Try it out

In [ ]:
program = """
(def rad 5)
rad
(def radiusfunc (func (radius) (* pi (* radius radius))))
(radiusfunc rad)
(def myvar 0)
(if (== myvar 1) (store rad 6) (store rad 7))
(radiusfunc rad)
(== 1 1)
"""

In [ ]:
for s in parse_program(program):
    print(s)

[]
['def', 'rad', 5]
rad
['def', 'radiusfunc', ['func', ['radius'], ['*', 'pi', ['*', 'radius', 'radius']]]]
['radiusfunc', 'rad']
['def', 'myvar', 0]
['if', ['==', 'myvar', 1], ['store', 'rad', 6], ['store', 'rad', 7]]
['radiusfunc', 'rad']
['==', 1, 1]
[]


In [ ]:
for result in run_program(program, globenv):
    print(backtolang(result))

> []
> ['def', 'rad', 5]
> rad
> ['def', 'radiusfunc', ['func', ['radius'], ['*', 'pi', ['*', 'radius', 'radius']]]]
> ['radiusfunc', 'rad']
in dispatch args are call (5,)
in call ['radius'] ['*', 'pi', ['*', 'radius', 'radius']] (5,)
> ['def', 'myvar', 0]
> ['if', ['==', 'myvar', 1], ['store', 'rad', 6], ['store', 'rad', 7]]
> ['radiusfunc', 'rad']
in dispatch args are call (7,)
in call ['radius'] ['*', 'pi', ['*', 'radius', 'radius']] (7,)
> ['==', 1, 1]
> []
nil
nil
5
nil
78.53981633974483
nil
nil
153.93804002589985
#t
nil


In [ ]:
repl(globenv)# to get out of the repl in the notebook just cause an exception like below

calc> (+ 1 1)
2
calc> fdfdf


NameError: fdfdf <<>> not found in Environment